In [20]:
import requests
import os
import json
import time
from datetime import datetime, timedelta

In [62]:
# handles
with open(os.path.join("..", "materials", "news-handles.txt"), "r") as f:
    handles = f.read().split("\n")

# token and endpoint
with open(os.path.join("D:/", "repos", "tokens", "twitter_bearer.txt"), 'r') as f:
    bearer_token = f.read()

search_url = "https://api.twitter.com/2/tweets/search/all"

# set start_time
#d = datetime.today() - timedelta(days=7)
#start_time = f"{str(d.date())}T00:00:00Z"
start_time = "2020-02-01T00:00:00Z"
end_time = "2021-12-08T00:00:00Z"

query_string = ' OR '.join([f"(from:{handle})" for handle in handles])

query_params = {'query': query_string,
                'tweet.fields': 'entities,public_metrics,created_at,referenced_tweets',
                'expansions': 'author_id',
                'user.fields': 'created_at,description,public_metrics,url,verified', 
                'max_results': 500,
                'start_time': start_time,
                'end_time': end_time}


In [63]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FullArchiveSearchPython"
    return r


def connect_to_endpoint(url, params):
    response = requests.get(search_url, auth=bearer_oauth, params=params)
    #print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def initial():
    json_response = connect_to_endpoint(search_url, query_params)
    return(json_response)

def continued(next_token):
    new_params = query_params.copy()
    new_params['next_token'] = next_token
    json_response = connect_to_endpoint(search_url, new_params)
    return(json_response)

data = initial()
#all_data = data.copy()
#all_data.pop('meta', None)

used_next_tokens = []
next_token = data.get('meta').get('next_token')

if next_token is not None:
    while True:
        time.sleep(1)
        data = continued(next_token)
        all_data['data'] = all_data.get('data') + data.get('data')
        all_data['includes']['users'] = all_data.get('includes').get('users') + data.get('includes').get('users')

        used_next_tokens.append(next_token)

        next_token = data.get('meta').get('next_token')

        if next_token is None:
            break

In [64]:
len(all_data.get('data'))

423803

In [65]:
all_data.get('data')[len(all_data.get('data'))-1]

{'entities': {'mentions': [{'start': 133,
    'end': 141,
    'username': 'mcjomcg',
    'id': '45915319'}],
  'urls': [{'start': 234,
    'end': 257,
    'url': 'https://t.co/90xZmvrgdq',
    'expanded_url': 'https://on.wsj.com/31epgYx',
    'display_url': 'on.wsj.com/31epgYx'},
   {'start': 258,
    'end': 281,
    'url': 'https://t.co/XhUy5PL0mH',
    'expanded_url': 'https://twitter.com/WSJ/status/1223395537778626562/video/1',
    'display_url': 'pic.twitter.com/XhUy5PL0mH'}],
  'hashtags': [{'start': 221, 'end': 233, 'tag': 'WSJWhatsNow'}],
  'annotations': [{'start': 4,
    'end': 21,
    'probability': 0.9135,
    'type': 'Organization',
    'normalized_text': 'Kansas City Chiefs'},
   {'start': 27,
    'end': 45,
    'probability': 0.7282,
    'type': 'Organization',
    'normalized_text': 'San Francisco 49ers'},
   {'start': 126,
    'end': 130,
    'probability': 0.8115,
    'type': 'Place',
    'normalized_text': 'Miami'},
   {'start': 209,
    'end': 218,
    'probability':

In [67]:
with open(os.path.join("..", "data", "tpb_tweets_news-outlets_20211208.json"), "w") as f:
    json.dump(all_data, f)